<div class="alert alert-block alert-success">
    <h1 style="margin-bottom:0.5cm"><b>Evaluate decoder</b></h1>
    <p style="margin-bottom:0.25cm">This code is used to evaluate a 'decoder' that can predict finger positions from electromyographic signals, based on angular finger position data and electromyographic data from specific forearm muscles.
</div>

## Import all necessary packages

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

from sklearn import metrics

## Linear Regressoion

### Load data

In [ ]:
data = pd.read_csv("data/ev_data15.csv")
data

### Delete first and last rows "without data"

In [ ]:
for i in range(0,2):
    data.drop(i, inplace=True)
#for i in range(460,464):
#    data.drop(i, inplace=True)
data

### EDA

In [ ]:
data.info()

In [ ]:
data.describe()

### Calculate metrics

#### For each finger

In [ ]:
def regression_metrics(actual_values, predicted_values):
    
    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(actual_values, predicted_values)
    
    # Standard Deviation of Absolute Error (STD_AE)
    absolute_errors = np.abs(np.array(actual_values) - np.array(predicted_values))
    std_dev_absolute_error = np.std(absolute_errors)
    std_ae = np.mean(std_dev_absolute_error)
    
    # Max and Min Absolute Error
    max_ae = np.max(absolute_errors)
    min_ae = np.min(absolute_errors)
    
    # Mean Squared Error (MSE)
    mse = mean_squared_error(actual_values, predicted_values)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)

    # R-squared (Coefficient of determination)
    r_squared = r2_score(actual_values, predicted_values)
    
    return mae, std_ae, max_ae, min_ae, mse, rmse, r_squared

In [ ]:
thumb_mae, thumb_std_ae, thumb_max_ae, thumb_min_ae, thumb_mse, thumb_rmse, thumb_r_squared = regression_metrics(data['thumb'].tolist(), data['thumb_pred'].tolist())
index_mae, index_std_ae, index_max_ae, index_min_ae, index_mse, index_rmse, index_r_squared = regression_metrics(data['index'].tolist(), data['index_pred'].tolist())
long_mae, long_std_ae, long_max_ae, long_min_ae, long_mse, long_rmse, long_r_squared = regression_metrics(data['long'].tolist(), data['long_pred'].tolist())
ring_mae, ring_std_ae, ring_max_ae, ring_min_ae, ring_mse, ring_rmse, ring_r_squared = regression_metrics(data['ring'].tolist(), data['ring_pred'].tolist())
small_mae, small_std_ae, small_max_ae, small_min_ae, small_mse, small_rmse, small_r_squared = regression_metrics(data['small'].tolist(), data['small_pred'].tolist())

In [ ]:
# Prepare data
finger_data = [
    ["Thumb", thumb_mae, thumb_std_ae, thumb_max_ae, thumb_min_ae, thumb_mse, thumb_rmse, thumb_r_squared],
    ["Index", index_mae, index_std_ae, index_max_ae, index_min_ae, index_mse, index_rmse, index_r_squared],
    ["Long", long_mae, long_std_ae, long_max_ae, long_min_ae, long_mse, long_rmse, long_r_squared],
    ["Ring", ring_mae, ring_std_ae, ring_max_ae, ring_min_ae, ring_mse, ring_rmse, ring_r_squared],
    ["Small", small_mae, small_std_ae, small_max_ae, small_min_ae, small_mse, small_rmse, small_r_squared]
]

# Print table
print("Angle error for each finger (in degrees)")
print(tabulate(finger_data, headers=["Finger", "MAE", "STD_AE", "MAX_AE", "MIN_AE", "MSE", "RMSE", "R-squared"], tablefmt="grid"))
print("  - MAE: Mean Absolute Error")
print("  - STD_AE: Standard Deviation of the Absolute Error")
print("  - MSE: Mean Squared Error")
print("  - RMSE: Root Mean Squared Error")
print("  - R-squared: Coefficient of determination")

#### In general

In [ ]:
mae = np.mean([thumb_mae, index_mae, long_mae, ring_mae, small_mae])
std_ae = np.mean([thumb_std_ae, index_std_ae, long_std_ae, ring_std_ae, small_std_ae])
max_ae = np.max([thumb_max_ae, index_max_ae, long_max_ae, ring_max_ae, small_max_ae])
min_ae = np.min([thumb_min_ae, index_min_ae, long_min_ae, ring_min_ae, small_min_ae])
mse = np.mean([thumb_mse, index_mse, long_mse, ring_mse, small_mse])
rmse = np.mean([thumb_rmse, index_rmse, long_rmse, ring_rmse, small_rmse])
r_squared = np.mean([thumb_r_squared, index_r_squared, long_r_squared, ring_r_squared, small_r_squared])

# Prepare data
general_data = [
    [mae, std_ae, max_ae, min_ae, mse, rmse, r_squared]
]

# Print table
print("Angle error ingeneral ==> mean (in degrees)")
print(tabulate(general_data, headers=["MAE", "STD_AE", "MAX_AE", "MIN_AE", "MSE", "RMSE", "R-squared"], tablefmt="grid"))
print("  - MAE: Mean Absolute Error")
print("  - STD_AE: Standard Deviation of the Absolute Error")
print("  - MSE: Mean Squared Error")
print("  - RMSE: Root Mean Squared Error")
print("  - R-squared: Coefficient of determination")

In [ ]:
nothumb_mae = np.mean([index_mae, long_mae, ring_mae, small_mae])
nothumb_std_ae = np.mean([index_std_ae, long_std_ae, ring_std_ae, small_std_ae])
nothumb_max_ae = np.max([index_max_ae, long_max_ae, ring_max_ae, small_max_ae])
nothumb_min_ae = np.min([index_min_ae, long_min_ae, ring_min_ae, small_min_ae])
nothumb_mse = np.mean([index_mse, long_mse, ring_mse, small_mse])
nothumb_rmse = np.mean([index_rmse, long_rmse, ring_rmse, small_rmse])
nothumb_r_squared = np.mean([index_r_squared, long_r_squared, ring_r_squared, small_r_squared])

# Prepare data
general_data = [
    [nothumb_mae, nothumb_std_ae, nothumb_max_ae, nothumb_min_ae, nothumb_mse, nothumb_rmse, nothumb_r_squared]
]

# Print table
print("Angle error ingeneral ==> mean (in degrees)")
print(tabulate(general_data, headers=["MAE", "STD_AE", "MAX_AE", "MIN_AE", "MSE", "RMSE", "R-squared"], tablefmt="grid"))
print("  - MAE: Mean Absolute Error")
print("  - STD_AE: Standard Deviation of the Absolute Error")
print("  - MSE: Mean Squared Error")
print("  - RMSE: Root Mean Squared Error")
print("  - R-squared: Coefficient of determination")

### Export data

In [ ]:
import csv

with open('S4_LR_test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['S4', 'MAE', 'STD_AE', 'MAX_AE', 'MIN_AE', 'MSE', 'RMSE', 'R-squared'])
    writer.writerow(['Thumb', thumb_mae, thumb_std_ae, thumb_max_ae, thumb_min_ae, thumb_mse, thumb_rmse, thumb_r_squared])
    writer.writerow(['Index', index_mae, index_std_ae, index_max_ae, index_min_ae, index_mse, index_rmse, index_r_squared])
    writer.writerow(['Long', long_mae, long_std_ae, long_max_ae, long_min_ae, long_mse, long_rmse, long_r_squared])
    writer.writerow(['Ring', ring_mae, ring_std_ae, ring_max_ae, ring_min_ae, ring_mse, ring_rmse, ring_r_squared])
    writer.writerow(['Small', small_mae, small_std_ae, small_max_ae, small_min_ae, small_mse, small_rmse, small_r_squared])
    writer.writerow(['General', mae, std_ae, max_ae, min_ae, mse, rmse, r_squared])
    writer.writerow(['General_nothumb', nothumb_mae, nothumb_std_ae, nothumb_max_ae, nothumb_min_ae, nothumb_mse, nothumb_rmse, nothumb_r_squared])

### Correlation

In [ ]:
thumb_corr = np.corrcoef(data['thumb'], data['thumb_pred'])[0, 1]
index_corr = np.corrcoef(data['index'], data['index_pred'])[0, 1]
long_corr = np.corrcoef(data['long'], data['long_pred'])[0, 1]
ring_corr = np.corrcoef(data['ring'], data['ring_pred'])[0, 1]
small_corr = np.corrcoef(data['small'], data['small_pred'])[0, 1]
general_corr = np.mean([thumb_corr, index_corr, long_corr, ring_corr, small_corr])
general_nothumb_corr = np.mean([index_corr, long_corr, ring_corr, small_corr])

print("thumb_corr: {:.2f}".format(thumb_corr))
print("index_corr: {:.2f}".format(index_corr))
print("long_corr: {:.2f}".format(long_corr))
print("ring_corr: {:.2f}".format(ring_corr))
print("small_corr: {:.2f}".format(small_corr))
print("general_corr: {:.2f}".format(general_corr))
print("general_nothumb_corr: {:.2f}".format(general_nothumb_corr))

In [ ]:
import csv

with open('S4_LR_test_corr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['S', 'corr'])
    writer.writerow(['Thumb', thumb_corr])
    writer.writerow(['Index', index_corr])
    writer.writerow(['Long', long_corr])
    writer.writerow(['Ring', ring_corr])
    writer.writerow(['Small', small_corr])
    writer.writerow(['General', general_corr])
    writer.writerow(['General_nothumb', general_nothumb_corr])

### Plot vectors

In [ ]:
def plot_vector(y_values, color='b', label=None):
    x_values = range(len(y_values))
    plt.plot(x_values, y_values, color=color, label=label)
    plt.xlabel('Index')
    plt.ylabel('Y Values')
    plt.grid(True)
    plt.legend()

#### Thumb

In [ ]:
plot_vector(data['thumb'], color='r', label='Test')
plot_vector(data['thumb_pred'], color='g', label='Pred')
plt.title('Thumb')
plt.show()

In [ ]:
# Calculate the correlation coefficient
correlation_coefficient = np.corrcoef(data['thumb'], data['thumb_pred'])[0, 1]

print("Correlation coefficient: {:.2f}".format(correlation_coefficient))

#### Index

In [ ]:
plot_vector(data['index'], color='r', label='Test')
plot_vector(data['index_pred'], color='g', label='Pred')
plt.title('Index')
plt.show()

In [ ]:
# Calculate the correlation coefficient
correlation_coefficient = np.corrcoef(data['index'], data['index_pred'])[0, 1]

print("Correlation coefficient: {:.2f}".format(correlation_coefficient))

#### Long

In [ ]:
plot_vector(data['long'], color='r', label='Test')
plot_vector(data['long_pred'], color='g', label='Pred')
plt.title('Long')
plt.show()

In [ ]:
# Calculate the correlation coefficient
correlation_coefficient = np.corrcoef(data['long'], data['long_pred'])[0, 1]

print("Correlation coefficient: {:.2f}".format(correlation_coefficient))

#### Ring

In [ ]:
plot_vector(data['ring'], color='r', label='Test')
plot_vector(data['ring_pred'], color='g', label='Pred')
plt.title('Ring')
plt.show()

In [ ]:
# Calculate the correlation coefficient
correlation_coefficient = np.corrcoef(data['ring'], data['ring_pred'])[0, 1]

print("Correlation coefficient: {:.2f}".format(correlation_coefficient))

#### Small

In [ ]:
plot_vector(data['small'], color='r', label='Test')
plot_vector(data['small_pred'], color='g', label='Pred')
plt.title('Small')
plt.show()

In [ ]:
# Calculate the correlation coefficient
correlation_coefficient = np.corrcoef(data['small'], data['small_pred'])[0, 1]

print("Correlation coefficient: {:.2f}".format(correlation_coefficient))

## LDA

### Load data

In [ ]:
data = pd.read_csv("data/ev_data16.csv")
data

### Delete first and last rows "without data"

In [ ]:
for i in range(0,5):
    data.drop(i, inplace=True)
#for i in range(460,464):
#    data.drop(i, inplace=True)
data

### Preprocessing for classification

In [ ]:
data['thumb_bool'] = (data['thumb'] > data['thumb'].mean()).astype(int)
data['index_bool'] = (data['index'] > data['index'].mean()).astype(int)
data['long_bool'] = (data['long'] > data['long'].mean()).astype(int)
data['ring_bool'] = (data['ring'] > data['ring'].mean()).astype(int)
data['small_bool'] = (data['small'] > data['small'].mean()).astype(int)

data['thumb_pred_bool'] = (data['thumb_pred'] > data['thumb_pred'].mean()).astype(int)
data['index_pred_bool'] = (data['index_pred'] > data['index_pred'].mean()).astype(int)
data['long_pred_bool'] = (data['long_pred'] > data['long_pred'].mean()).astype(int)
data['ring_pred_bool'] = (data['ring_pred'] > data['ring_pred'].mean()).astype(int)
data['small_pred_bool'] = (data['small_pred'] > data['small_pred'].mean()).astype(int)

data

### EDA

In [ ]:
data.info()

In [ ]:
data.describe()

### Scores

#### For each finger

In [ ]:
def metrics_classification(actual_values, predicted_values):
    
    # Calculate accuracy
    accuracy = accuracy_score(actual_values, predicted_values)

    # Calculate precision
    precision = precision_score(actual_values, predicted_values)

    # Calculate recall (sensitivity)
    recall = recall_score(actual_values, predicted_values)

    # Calculate F1-score
    f1 = f1_score(actual_values, predicted_values)
    
    return accuracy, precision, recall, f1

In [ ]:
thumb_accuracy, thumb_precision, thumb_recall, thumb_f1_score = metrics_classification(data['thumb_bool'].tolist(), data['thumb_pred_bool'].tolist())
index_accuracy, index_precision, index_recall, index_f1_score= metrics_classification(data['index_bool'].tolist(), data['index_pred_bool'].tolist())
long_accuracy, long_precision, long_recall, long_f1_score = metrics_classification(data['long_bool'].tolist(), data['long_pred_bool'].tolist())
ring_accuracy, ring_precision, ring_recall, ring_f1_score = metrics_classification(data['ring_bool'].tolist(), data['ring_pred_bool'].tolist())
small_accuracy, small_precision, small_recall, small_f1_score = metrics_classification(data['small_bool'].tolist(), data['small_pred_bool'].tolist())

In [ ]:
# Prepare data
finger_data = [
    ["Thumb", thumb_accuracy, thumb_precision, thumb_recall, thumb_f1_score],    
    ["Index", index_accuracy, index_precision, index_recall, index_f1_score],
    ["Long", long_accuracy, long_precision, long_recall, long_f1_score],
    ["Ring", ring_accuracy, ring_precision, ring_recall, ring_f1_score],
    ["Small", small_accuracy, small_precision, small_recall, small_f1_score]
]

# Print table
print("Scores for each finger")
print(tabulate(finger_data, headers=["Finger", "Accuracy", "Precision", "Recall", "F1-score"], tablefmt="grid"))

#### In general

In [ ]:
accuracy = np.mean([thumb_accuracy, index_accuracy, long_accuracy, ring_accuracy, small_accuracy])
precision = np.mean([thumb_precision, index_precision, long_precision, ring_precision, small_precision])
recall = np.max([thumb_recall, index_recall, long_recall, ring_recall, small_recall])
f1_score = np.min([thumb_f1_score, index_f1_score, long_f1_score, ring_f1_score, small_f1_score])

# Prepare data
finger_data = [
    [accuracy, precision, recall, f1_score]
]

# Print table
print("Scores in general")
print(tabulate(finger_data, headers=["Accuracy", "Precision", "Recall", "F1-score"], tablefmt="grid"))

In [ ]:
nothumb_accuracy = np.mean([index_accuracy, long_accuracy, ring_accuracy, small_accuracy])
nothumb_precision = np.mean([index_precision, long_precision, ring_precision, small_precision])
nothumb_recall = np.max([index_recall, long_recall, ring_recall, small_recall])
nothumb_f1_score = np.min([index_f1_score, long_f1_score, ring_f1_score, small_f1_score])

# Prepare data
finger_data = [
    [nothumb_accuracy, nothumb_precision, nothumb_recall, nothumb_f1_score]
]

# Print table
print("Scores in general")
print(tabulate(finger_data, headers=["Accuracy", "Precision", "Recall", "F1-score"], tablefmt="grid"))

### Export data

In [ ]:
import csv

with open('S4_LDA.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['S4', 'Accuracy', 'Precision', 'Recall', 'F1-score'])
    writer.writerow(['Thumb', thumb_accuracy, thumb_precision, thumb_recall, thumb_f1_score])
    writer.writerow(['Index', index_accuracy, index_precision, index_recall, index_f1_score])
    writer.writerow(['Long', long_accuracy, long_precision, long_recall, long_f1_score])
    writer.writerow(['Ring', ring_accuracy, ring_precision, ring_recall, ring_f1_score])
    writer.writerow(['Small', ring_accuracy, ring_precision, ring_recall, ring_f1_score])
    writer.writerow(['General', accuracy, precision, recall, f1_score])
    writer.writerow(['General_nothumb', nothumb_accuracy, nothumb_precision, nothumb_recall, nothumb_f1_score])

### Confussion matrix

In [ ]:
def confussion_matrix(actual_values, predicted_values):
    # Calculate confusion matrix
    conf_matrix = confusion_matrix(actual_values, predicted_values)

    # Define labels for the confusion matrix
    labels = ['True Negative', 'False Positive', 'False Negative', 'True Positive']

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title('Confusion Matrix')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.show()
    
    return conf_matrix

#### Thumb

In [ ]:
thumb_conf_matrix = confussion_matrix(data['thumb_bool'].tolist(), data['thumb_pred_bool'].tolist())

#### Index

In [ ]:
index_conf_matrix = confussion_matrix(data['index_bool'].tolist(), data['index_pred_bool'].tolist())

#### Long

In [ ]:
long_conf_matrix = confussion_matrix(data['long_bool'].tolist(), data['long_pred_bool'].tolist())

#### Ring

In [ ]:
ring_conf_matrix = confussion_matrix(data['ring_bool'].tolist(), data['ring_pred_bool'].tolist())

#### Small

In [ ]:
small_conf_matrix = confussion_matrix(data['small_bool'].tolist(), data['small_pred_bool'].tolist())

### EXTRA
Esto es para sacar la matriz de confusión a mano (no hace falta):

In [ ]:
TN = 0 # True Negative
TP = 0 # True Positive
FN = 0 # False Negative
FP = 0 # False Positive

for index, row in data.iterrows():
    ref = row['thumb_bool']
    pred = row['thumb_pred_bool']
    
    # Perform comparison
    if ref == 0 and pred == 0:
        TN = TN + 1
    elif ref == 1 and pred == 1:
        TP = TP + 1
    elif ref == 0 and pred == 1:
        FP = FP + 1
    elif ref == 1 and pred == 0:
        FN = FN + 1
        
# Print the values in a 2x2 table with row and column names
print("+----------+----------+---------------------+")
print("|{:<10}|{:<10}|{:<20}|".format("", "", "      Predicted      "))
print("+----------+----------+---------------------+")
print("|{:<10}|{:<10}|{:<10}|{:<10}|".format("", "", " Positive", " Negative"))
print("+----------+----------+---------------------+")
print("|{:<10}|{:<10}|{:<10}|{:<10}|".format("  Actual", " Positive", TP, FN))
print("+          +----------+---------------------+")
print("|{:<10}|{:<10}|{:<10}|{:<10}|".format("", " Negative", FP, TN))
print("+----------+----------+---------------------+")